This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [2]:
import pandas as pd # standard naming convention
import numpy as np

N = 1000000
dataframe = pd.read_csv('data_000637.txt',nrows=N)
dataframe.head()

print(dataframe)

        HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0          1     0          123  3869200167        2374        26
1          1     0          124  3869200167        2374        27
2          1     0           63  3869200167        2553        28
3          1     0           64  3869200167        2558        19
4          1     0           64  3869200167        2760        25
...      ...   ...          ...         ...         ...       ...
999995     1     0           62  3869209011        2560        29
999996     1     0           51  3869209011        2561        12
999997     1     0           61  3869209011        2562         2
999998     1     0           50  3869209011        2563         4
999999     1     0           60  3869209011        2688        25

[1000000 rows x 6 columns]


2\. Estimate the number of BX in a ORBIT (the value 'x').

In [3]:
dataframe.sort_values(by="BX_COUNTER", ascending=False)
print(dataframe)
BX = dataframe["BX_COUNTER"]
#print(BX)
max = 0
for i in BX:
    if i > max:
        max = i
max += 1
print("x =",max)

        HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0          1     0          123  3869200167        2374        26
1          1     0          124  3869200167        2374        27
2          1     0           63  3869200167        2553        28
3          1     0           64  3869200167        2558        19
4          1     0           64  3869200167        2760        25
...      ...   ...          ...         ...         ...       ...
999995     1     0           62  3869209011        2560        29
999996     1     0           51  3869209011        2561        12
999997     1     0           61  3869209011        2562         2
999998     1     0           50  3869209011        2563         4
999999     1     0           60  3869209011        2688        25

[1000000 rows x 6 columns]
x = 3564


3\. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

In [4]:
#Each TDC count corresponds to 25/30 ns
#A unit of BX_COUNTER corresponds to 25 ns 
#The ORBIT_CNT is increased every 'x' BX_COUNTER

import datetime

X = 3563

nrow = 0
row_init = dataframe.iloc[nrow]
nanos_init = row_init['TDC_MEAS']*25/30 + row_init['BX_COUNTER']*25 + row_init['ORBIT_CNT']*X*25
                                             
sec_init = nanos_init *10**-9
print(row_init)
print("Duration of the data taking of N sample:")
print(str(datetime.timedelta(seconds=sec_init)))


HEAD                    1
FPGA                    0
TDC_CHANNEL           123
ORBIT_CNT      3869200167
BX_COUNTER           2374
TDC_MEAS               26
Name: 0, dtype: int64
Duration of the data taking of N sample:
3 days, 23:44:09.004935


4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

In [5]:
X = 3563
dataframe['Row Time'] =  (dataframe['TDC_MEAS']*25/30 + dataframe['BX_COUNTER']*25 + dataframe['ORBIT_CNT']*X*25).cumsum()
print(dataframe)

        HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS  \
0          1     0          123  3869200167        2374        26   
1          1     0          124  3869200167        2374        27   
2          1     0           63  3869200167        2553        28   
3          1     0           64  3869200167        2558        19   
4          1     0           64  3869200167        2760        25   
...      ...   ...          ...         ...         ...       ...   
999995     1     0           62  3869209011        2560        29   
999996     1     0           51  3869209011        2561        12   
999997     1     0           61  3869209011        2562         2   
999998     1     0           50  3869209011        2563         4   
999999     1     0           60  3869209011        2688        25   

            Row Time  
0       3.446490e+14  
1       6.892980e+14  
2       1.033947e+15  
3       1.378596e+15  
4       1.723245e+15  
...              ...  
999995  3.

5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [6]:
df_maxTDC = dataframe.groupby(['TDC_MEAS'], as_index=False).max()
print(df_maxTDC.iloc[-3:])

    TDC_MEAS  HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER      Row Time
27        27     1     1          128  3869209011        3563  3.446477e+20
28        28     1     1          130  3869209011        3563  3.446377e+20
29        29     1     1          128  3869209011        3563  3.446481e+20


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [7]:
df1 = dataframe[dataframe.isna().any(axis=1)]
print('Number of non-empty orbits: ', df1.shape[0])

df_ORBIT = dataframe.groupby(['ORBIT_CNT'], as_index=False)['TDC_CHANNEL'].max()
print('Number of ORBITS_CNT with TDC_CHANNEL=139: ', df_ORBIT.shape[0])

Number of non-empty orbits:  0
Number of ORBITS_CNT with TDC_CHANNEL=139:  8841


7\. **Optional:** Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel